In [22]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")

pd.set_option('display.max_columns', None)
#print(train.columns)

In [23]:
IDCOL = 'Id'
target = "SalePrice"
SEED = 42

In [24]:
train = train.drop(IDCOL, axis=1)
test  = test.drop(IDCOL, axis=1)

# Для числових колонок — замінимо median
numeric_cols = train.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    if col == "SalePrice":
        continue  # пропускаємо цільову змінну
    median = train[col].median()
    train[col] = train[col].fillna(median)
    test[col] = test[col].fillna(median)
    
# Для категоріальних колонок — замінимо на "None"
for col in train.select_dtypes(include=['object']).columns:
    train[col] = train[col].fillna("None")
    test[col] = test[col].fillna("None")

# Label Encoding для категоріальних фіч
cat_cols = train.select_dtypes(include=['object']).columns
for col in cat_cols:
    le = LabelEncoder()
    le.fit(list(train[col]) + list(test[col]))  # щоб коди були узгоджені
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

print("✅ Дані готові. Розмір train:", train.shape, "test:", test.shape)

✅ Дані готові. Розмір train: (1460, 80) test: (1459, 79)


In [25]:

# Загальна площа (одна з найсильніших ознак)
train["TotalSF"] = train["TotalBsmtSF"] + train["1stFlrSF"] + train["2ndFlrSF"]
test["TotalSF"]  = test["TotalBsmtSF"] + test["1stFlrSF"] + test["2ndFlrSF"]

# Загальна кількість ванних кімнат
train["TotalBath"] = train["FullBath"] + 0.5 * train["HalfBath"] + \
                     train["BsmtFullBath"] + 0.5 * train["BsmtHalfBath"]
test["TotalBath"] = test["FullBath"] + 0.5 * test["HalfBath"] + \
                    test["BsmtFullBath"] + 0.5 * test["BsmtHalfBath"]

# Вік будинку на момент продажу
train["HouseAge"] = train["YrSold"] - train["YearBuilt"]
test["HouseAge"]  = test["YrSold"] - test["YearBuilt"]

# Вік після останнього ремонту
train["RemodAge"] = train["YrSold"] - train["YearRemodAdd"]
test["RemodAge"]  = test["YrSold"] - test["YearRemodAdd"]

# Є гараж чи ні (бінарна)
train["HasGarage"] = (train["GarageArea"] > 0).astype(int)
test["HasGarage"]  = (test["GarageArea"] > 0).astype(int)

# Є підвал
train["HasBsmt"] = (train["TotalBsmtSF"] > 0).astype(int)
test["HasBsmt"]  = (test["TotalBsmtSF"] > 0).astype(int)

# Є камін
train["HasFireplace"] = (train["Fireplaces"] > 0).astype(int)
test["HasFireplace"]  = (test["Fireplaces"] > 0).astype(int)

# Загальна площа житлових приміщень
train["TotalArea"] = train["GrLivArea"] + train["TotalBsmtSF"] + train["GarageArea"]
test["TotalArea"]  = test["GrLivArea"] + test["TotalBsmtSF"] + test["GarageArea"]

# Співвідношення ціна / площа (у train, не в test)
train["PricePerSF"] = train["SalePrice"] / train["TotalSF"]

# Додатковий показник якості житла (комбінує декілька ознак)
train["OverallQualCond"] = train["OverallQual"] * train["OverallCond"]
test["OverallQualCond"]  = test["OverallQual"] * test["OverallCond"]

# Співвідношення житлової площі до ділянки
train["LivLotRatio"] = train["GrLivArea"] / (train["LotArea"] + 1)
test["LivLotRatio"]  = test["GrLivArea"] / (test["LotArea"] + 1)

# Співвідношення площі 1 поверху до всіх поверхів
train["1st2ndRatio"] = train["1stFlrSF"] / (train["2ndFlrSF"] + 1)
test["1st2ndRatio"]  = test["1stFlrSF"] / (test["2ndFlrSF"] + 1)

# Загальна якість житла
train["TotalQual"] = train["OverallQual"] + train["OverallCond"]
test["TotalQual"]  = test["OverallQual"] + test["OverallCond"]

# Відношення житлової площі до поверхні ділянки
train["LivingRatio"] = train["GrLivArea"] / train["LotArea"]
test["LivingRatio"]  = test["GrLivArea"] / test["LotArea"]

# Взаємодія якості і площі
train["QualArea"] = train["OverallQual"] * train["GrLivArea"]
test["QualArea"]  = test["OverallQual"] * test["GrLivArea"]

# Чи був будинок побудований або ремонтувався за останні 10 років
train["IsNew"] = ((train["YrSold"] - train["YearBuilt"]) < 10).astype(int)
test["IsNew"]  = ((test["YrSold"] - test["YearBuilt"]) < 10).astype(int)


train["SalePrice"] = np.log1p(train["SalePrice"])

X = train.drop(["SalePrice", "PricePerSF"], axis=1)
y = train["SalePrice"]
X_test = test


In [28]:
'''param_grid = {
    'n_estimators': [500],
    'max_depth': [2],
    'learning_rate':[0.05],
    'subsample':[0.8],         
    'colsample_bytree':[0.8]
}'''

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=SEED)


model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="rmse",               # тепер тут!
    early_stopping_rounds=100,
    random_state=SEED
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=100
)
y_train_pred = model.predict(X)

'''grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # метрика RMSE
    cv=3,                                   # 3-фолд крос-валідація
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X, y)'''

# Найкращі параметри та CV скор
#print("Best parameters:", grid_search.best_params_)
#print("Best CV accuracy:", grid_search.best_score_)
# Прогнози для тесту
#y_test_pred = grid_search.best_estimator_.predict(X_test)
# Зворотне логарифмування
#y_test_pred = np.expm1(y_test_pred)

[0]	validation_0-rmse:0.42667
[100]	validation_0-rmse:0.11893
[200]	validation_0-rmse:0.11339
[300]	validation_0-rmse:0.11339
[318]	validation_0-rmse:0.11308


"grid_search = GridSearchCV(\n    estimator=model,\n    param_grid=param_grid,\n    scoring='neg_root_mean_squared_error',  # метрика RMSE\n    cv=3,                                   # 3-фолд крос-валідація\n    verbose=2,\n    n_jobs=-1\n)\n\ngrid_search.fit(X, y)"

In [ ]:
#y_train_pred = grid_search.best_estimator_.predict(X)
#rmse = np.sqrt(mean_squared_error(y, y_train_pred))
#print(f"✅ Train RMSE: {rmse:.4f}")
y_train_pred = model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_train_pred))
print(f"✅ Train RMSE: {rmse:.4f}")
y_test_pred = model.predict(X_test)
# Якщо робили np.log1p на SalePrice
y_test_pred = np.expm1(y_test_pred)

In [ ]:
# зчитуємо оригінальний тест, щоб узяти колонку Id
submission = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
submission['SalePrice'] = y_test_pred
submission[['Id', 'SalePrice']].to_csv('submission.csv', index=False)
print("🚀 Submission файл створено успішно!")